# Amazon Web Scrapping with Selenium and Beautiful Soup

In [3]:
#Importing libraries
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

#Get url based on search term and Navigating to next page
def get_url(search_term):
    template='https://www.amazon.com/s?k={}&crid=25VEVPPHRCLCX'
    search_term=search_term.replace(' ','+')
    url=template.format(search_term)
    url+='&page={}'
    return url

#Extracting each search item as a dataframe within a method
def extract_record(item):
    atag=item.h2.a
    description=atag.text.strip()
    url='https://www.amazon.com'+atag.get('href')
    try:
        price_parent=item.find('span','a-price')
        price=price_parent.find('span','a-offscreen').text
    except AttributeError:
            return
    try:
        rating=item.i.text
        review_count=item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        rating=''
        review_count=0
    df=pd.DataFrame(columns=['Description','price','rating','review_count','url'])
    df=df.append({'Description': description,  'price': price,'rating':rating,'review_count':review_count,'url':url}, ignore_index=True)
    return df

#Main function initiate webdriver and invoke methods
def main(search_term):
    """Run main program"""
    #Start webdriver
    driver=webdriver.Chrome('C://Users//Hp//Downloads//chromedriver_win32 (1)//chromedriver.exe')
    url=get_url(search_term)
    df=pd.DataFrame(columns=['Description','price','rating','review_count','url'])
    for page in range(1,21):
        driver.get(url.format(page))
        soup=BeautifulSoup(driver.page_source,'html.parser')
        results=soup.find_all('div',{'data-component-type':'s-search-result'})
        for item in results:
            df=df.append(extract_record(item))
        print(df)
    driver.close()
    
    #Saving results to csv file
    df.to_csv("results.csv",encoding='utf-8',header=True,index=False)
       

In [5]:
main('mobile phones')

                                         Description    price  \
0  Total Wireless TCL A2 X 4G LTE Prepaid Smartph...   $76.89   
0  Tello Mobile - Bring Your Own Phone - 3 in 1 G...    $2.00   
0  LG K40 X420 32GB Unlocked GSM Phone w/ 13MP Ca...  $128.98   
0  Samsung Electronics Galaxy A42 5G, Factory Unl...  $399.99   
0  OnePlus 8 Glacial Green,​ 5G Unlocked Android ...  $398.69   
0  Samsung Electronics Galaxy A12, Factory Unlock...  $179.99   
0  Latest Android 11 Phone, Ulefone Note 6 Unlock...   $88.99   
0  Motorola One 5G Ace | 2021 | 2-Day battery | U...  $359.00   
0  Car Phone Holder Mount, 2021 Upgraded Vent Cli...   $19.99   
0  Moto G stylus | 2020 | Unlocked | Made for US ...  $229.95   
0  TCL 10 Pro Unlocked Android Smartphone with 6....  $399.99   
0  (1) Font Mobile Phone Holder, Compact and Port...    $6.99   
0  TCL 20S Unlocked Android Smartphone with 6.67”...  $249.99   
0  Nokia 3.4 | Android 10 | Unlocked Smartphone |...  $179.99   
0  Blackview Android Phon